In [1]:
import pandas as pd

df = pd.read_csv('https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv')
df.set_index('Object ID')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Object Number,Is Highlight,Is Public Domain,Department,Object Name,Title,Culture,Period,Dynasty,Reign,...,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository
Object ID,,,,,,,,,,,,,,,,,,,,,
1,1979.486.1,False,False,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
2,1980.264.5,False,False,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
3,67.265.9,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
4,67.265.10,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
5,67.265.11,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
6,67.265.12,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/6,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
7,67.265.13,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/7,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
8,67.265.14,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/8,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"
9,67.265.15,False,False,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/9,11/26/2018 8:00:04 AM,"Metropolitan Museum of Art, New York, NY"


In [2]:
df.columns

Index(['Object Number', 'Is Highlight', 'Is Public Domain', 'Object ID',
       'Department', 'Object Name', 'Title', 'Culture', 'Period', 'Dynasty',
       'Reign', 'Portfolio', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Metadata Date',
       'Repository'],
      dtype='object')

In [3]:
df=df[['Object ID','Department','Object Name','Title','Artist Display Name','Object Date','Country','Link Resource']]

In [4]:
import re
import string
import numpy as np

def format_replace_words(replace_dict):
    format_replace_dict = dict()
    for k,v in replace_dict.items():
        format_replace_dict[k.lower()] = v
    return format_replace_dict

def multiword_replace(text, replacement_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(replacement_dict,key=len,reverse=True)]), re.I)
    new_text = re.sub(pattern, lambda x: replacement_dict[x.group(0)], text)
    return new_text

def multiregex_replace(text, regex_dict):
    for k,v in regex_dict.items():
        regex = re.compile(k, re.I)
        match = re.search(regex, text)
        if match:
            text = re.sub(regex, v, text)
    return text

def lc_match_group(obj):
    return obj.group().lower()

def formatted_raw_string(text):
    if text is not None:
        text = text.title()
        format_pattern = r"(?<=[a-z])[\']([A-Z])"
        text = re.sub(format_pattern, lc_match_group, text)
    return text

def clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern):
    df[category] = df[category].astype(str)
    df[category] = np.where(df[category].str.contains(replace_pattern, na = False, case=False, regex=True) == True,
                            df[category].str.replace(replace_pattern, '', flags=re.I),
                            (np.where(df[category].str.contains(removal_pattern, na = False, case=False, regex=True) == True,
                                     'nan',
                                     df[category])))
    df[category] = np.where((np.char.find(df[category].str.lower().values.astype(str),"?")!=-1),'nan',df[category])
    
    compiled_list = list(df[category])
    for x in range(len(compiled_list)): 
        compiled_list[x] = " ".join(compiled_list[x].lower().split())
        compiled_list[x] = multiword_replace(compiled_list[x], word_dict)
        compiled_list[x] = multiregex_replace(compiled_list[x], regex_dict)
        compiled_list[x] = formatted_raw_string(compiled_list[x])
        token_list = compiled_list[x].split("|")
        for y in range(len(token_list)):
            token_list[y] = token_list[y].strip()
        token_list = sorted(filter(None, token_list))
        token_list = set(token_list)
        compiled_list[x] = ' | '.join(token_list)
    df[category] = pd.Series(compiled_list)
    df[category] = np.where(df[category] == 'Nan', np.nan, df[category])

In [5]:
def clean_country_data(df):
    
    category = 'Country'
    word_dict = {
        'Algieria': 'Algeria',
        'Republic of Congo': 'Republic Of The Congo',
        'yugoslavia0' : 'Yugoslavia)',
        'Dem. Rep. Congo': 'Democratic Republic Of The Congo',
        '- Turkey': 'Turkey'
    }
    word_dict = format_replace_words(word_dict)
    
    regex_dict = {
        #Inside content text has to be lowercase
        '\s+and\s+': '|',
        '\s*/\s*': '|',
        '^uk$': 'United Kingdom',
        '^u[.]*s[.]*[a.]*': 'United States',
        '\s*&\s*': '|'
    }
    
    replace_pattern = r'[(]*north[)]*\w*|south\w*|west\w*|east\w*|central|nothern|sothern|:\s*n.a.|:n.|harold copping|bernard turner'
    removal_pattern = r'probably|probbly|possibly|perhaps|unknown|africa|asia|europe|america|middle east|mediterranean|lisez|flanders|[^A-Z]+or[^A-Z]+'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)
    

In [6]:
def clean_artist_name_data(df):
    
    category = 'Artist Display Name'
    word_dict = {
        ', and': ',',
        'Benjamin Bakewell & Co.': 'Bakewell, Page & Bakewell',
        'Bakewell, Page & Bakewells': 'Bakewell, Page & Bakewell',
    }
    
    regex_dict = {
        #Inside content text has to be lowercase
        '\s*[/;]\s*': '|',
        '^[A-Z].\s*[A-Z].\s*$': '',
        '^[A-Z].\s*[A-Z].\s*[A-Z].$': '',
        '^A$': '',
    }
    
    word_dict = format_replace_words(word_dict)
    replace_pattern = r'\d+[-]\d+|[.,]{1} paris|[.,]{1} berlin|[.,]{1} new york|[.,]{1} london|[.,]{1} england|"L"'
    removal_pattern = r'probably|prob.|possibly|perhaps|unknown|anonymous|painter|artist|[^A-Z]+or[^A-Z]+|^[A-Z].\s*[A-Z].\s*$|^[A-Z].\s*[A-Z].\s*[A-Z].$'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)

In [7]:
def clean_art_title_data(df):
    
    category = 'Title'
    word_dict = {
        'coffee cup saucer': 'Coffee Cup And Saucer',
        'doorknob': 'Door Knob',
        "\'": "'"
    }
    
    regex_dict = {
        #Inside content text has to be lowercase
        'chest[-]*\s*\w+[-]*\s*drawer[s]*': 'Chest With Drawer',
        '\s*[/]\s*': '/',
    }
    
    word_dict = format_replace_words(word_dict)
    replace_pattern = r'covered|^[[]|[]]$'
    removal_pattern = r'probably|prob.|possibly|perhaps|unknown|anonymous|[^A-Z]+or[^A-Z]+|'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)

In [8]:
country_list = list(df['Country'].astype(str).unique())
south_list = []
for x in country_list:
    if 'uk' in x.lower():
        south_list.append(x)
south_list

['Ukraine', 'Jordan, ecavated at Khirbat al-Mukhayyat', 'UK']

In [9]:
artist_name_list = list(df['Artist Display Name'].astype(str).unique())
check_list = []
for x in artist_name_list:
    if ';' in x.lower():
        check_list.append(x)
check_list

['Fourteen identified German (Augsburg) goldsmiths and other German artisans; Japanese (Imari) porcelain maker',
 'Guigoni; Bossi',
 'Northern Alaskan Eskimo; Family: Eskimavan',
 'Hopi Tribe; Shoshonean family']

In [11]:
title_list = list(df['Title'].astype(str).unique())
title_check_list = []
for x in title_list:
    if "\\'" in x.lower():
        title_check_list.append(x)
title_check_list

[]

In [12]:
def get_valid_id_list(category):
    
    #Returns list of object IDs that have valid (non-NaN) value in that particular category column
    #Category could be art title, artist name, country, time period, etc.
    import numpy as np
    
    valid_id_list = []
    valid_rows = np.where(df[category].notnull())[0].tolist()
    for x in valid_rows:
        valid_id_list.append(object_id_list[x])
        
    return valid_id_list

In [13]:
def get_wrong_answers(category, chosen_art):
    
    #Get list of Object IDs that have valid (non NaN) value for that category and have the
    #same department (relevant value) as the correct answer
    import numpy as np
    
    dept_valid_id_list = []
    department = df.loc[df['Object ID'] == chosen_art]['Department'].values[0]
    chosen_art_category_value = df.loc[df['Object ID'] == chosen_art][category].values[0]
    dept_valid_rows = np.where((df['Department'] == department) & (df[category] != chosen_art_category_value))[0].tolist()
    for x in dept_valid_rows:
        dept_valid_id_list.append(object_id_list[x])
    relevant_valid_id_list = list(set(dept_valid_id_list).intersection(set(valid_id_dict[category])))
    
    #Get randomy selected wrong answers among the relevant, valid ID list
    import random
    
    w1 = random.choice(relevant_valid_id_list)
    w2 = random.choice(relevant_valid_id_list)
    w3 = random.choice(relevant_valid_id_list)                                                                   
                                                                       
    return w1,w2,w3                         

In [14]:
def choose_category():
    
    import random
    
    category = random.choice(category_list)
    return category

In [15]:
def choose_art(valid_id_list):
    
    import random
    
    random_object_id = random.choice(valid_id_list)
    
    return random_object_id

In [16]:
def generate_question():
    
    category = choose_category()
    chosen_art = choose_art(valid_id_dict[category])
    w1,w2,w3 = get_wrong_answers(category, chosen_art)
    
    return category, chosen_art, w1, w2, w3

In [24]:
def clean_object_date(df):
    def extract_non_numeric(object_date):
        string=str(object_date)
        import re
        pattern=r'\D+'
        match=re.findall(pattern,string)
        return tuple(match)
    df['object_date_non_numeric']=df['Object Date'].apply(extract_non_numeric)
    def clean_object_date(date):
        import re
        def extract_year(string):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if 3<=len(m)<=4 and int(m)<=2018:
                return int(m[:-2])+1
            elif 1<=len(m)<=2:
                return 1
            else:
                return None
        century=None
        if date==None:
            century=None
        string=str(date)
        if string=='':
            century=None
        match=tuple(re.findall(r'\D+',string))
        if  match==('th century',) or match==('th–','th century') or match==('early ','th century') or match==('late ','th century') or match==('late ','th–early ','th century') or match==('mid-','th century') or match==('second half ','th century') or match==('first half ','th century') or match==('th–early ','th century'):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if not m==None:
                century=int(m)
        if match==('th century B.C.',) or match==('ca. ','th–','th century B.C.') or match==('th–','th century B.C.'):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if not m==None:
                century=-int(m)
        if match==('ca. ','–',' B.C.') or match==('ca. ',' B.C.') or match==('B.C.–A.D. ',):
            century=extract_year(string)
        if match==tuple() or match==('–',) or match==('s',) or match==('s–','s') or match==('fall/winter ','–') or match==('ca. ','–') or match==('ca. ',) or match==('dated ',) or match==('spring/summer ',):
            century=extract_year(string)
        return century
    df['century']=df['Object Date'].apply(clean_object_date)
    df.drop(['Object Date','object_date_non_numeric'],axis=1,inplace=True)

In [17]:
#Clean Data
clean_country_data(df)

In [18]:
clean_artist_name_data(df)

In [19]:
clean_art_title_data(df)

In [25]:
clean_object_date(df)

In [26]:
df.to_csv('C:/Users/向南/Desktop/MSBA/Tools for Analytics/clean_data.csv')

In [65]:
list(df['Country'].unique())

[nan,
 'Mexico',
 'Guatemala',
 'United States',
 'England',
 'Spain',
 'United States, England',
 'Netherlands',
 'France',
 'Canada',
 'United States, France',
 'England, France',
 'China',
 'Italy',
 'Japan',
 'Germany',
 'Czech Republic',
 'England, Ireland',
 'United States, Ireland',
 'Venezuela',
 'England, Netherlands',
 'England, Belgium',
 'Italy, France',
 'Bermuda',
 'Ireland',
 'India',
 'New Zealand Rotorua Whakarewarewa, France',
 'New Zealand Rotorua Whakarewarewa, China',
 'Italy, Germany',
 'Czech Republic, England',
 'France, Belgium',
 'France, England, Belgium',
 'United States, Germany',
 'Germany, France',
 'Scotland',
 'Czech Republic, Germany',
 'Holland',
 'United States, Denmark',
 'United States, Denmark, Finland',
 'United States, China',
 'India, China',
 'United States, Wales',
 'Italy, United States',
 'United States, England, Wales, Scotland',
 'Switzerland',
 'Portugal',
 'England, Scotland',
 'Paraguay',
 'Switzerland, France',
 'United States, France

In [ ]:
list(df['Artist Display Name'].unique())

In [ ]:
list(df['Title'].unique())

In [ ]:
list(df['Department'].unique())

In [42]:
#Initialize valid ids for all categories
category_list = ['Artist Display Name','Period','Title','Country','Department']
object_id_list = list(df['Object ID'])
valid_id_dict = dict()
for x in category_list:
    valid_id_dict[x] = get_valid_id_list(x)

In [53]:
active = True

# while active:
    
category,chosen_art,w1,w2,w3 = generate_question()
print(category,chosen_art,w1,w2,w3)

#Sometimes get an error that says "Cannot choose from an empty sequence" like the following.
#Need to find out what is causing this error and fix this.

IndexError: Cannot choose from an empty sequence